In [1]:
import palimpzest as pz
import pandas as pd
import time
import os
import IPython

formatter = IPython.get_ipython().display_formatter.formatters['text/plain']
formatter.max_seq_length = 0

# set OPENAI_API_KEY environment variable based on OPENAI_API_KEY
# os.environ["OPENAI_API_KEY"] = "{{ OPENAI_API_KEY }}" 

sci_paper_cols = [
    {"name": "title", "type": str, "desc": "The title of the paper. This is a natural language title, not a number or letter."},
    {"name": "author", "type": str, "desc": "The name of the first author of the paper"},
    {"name": "abstract", "type": str, "desc": "A short description of the paper contributions and findings"},
]

reference_cols = [
    {"name": "index", "type": int, "desc": "The index of the reference in the paper"},
    {"name": "title", "type": str, "desc": "The title of the paper being cited"},
    {"name": "first_author", "type": str, "desc": "The author of the paper being cited"},
    {"name": "year", "type": int, "desc": "The year in which the cited paper was published"},
]

print("Setup complete")

Setup complete


What are the datasets available in the system?

In [2]:
import pandas as pd
from prettytable import PrettyTable

dataset_path = "/home/gerardo/bdf-pz/testdata"
ds = os.listdir(dataset_path)

# construct table for printing
records = [["Name", "Path", "N. Files"]]
for path in ds:
    abspath = os.path.join(dataset_path,path)
    records.append([path,abspath,len(os.listdir(abspath))])

# print table of registered datasets
t = PrettyTable(records[0])
t.add_rows(records[1:])
t

Name,Path,N. Files
sigmod-tiny-txt,/home/gerardo/bdf-pz/testdata/sigmod-tiny-txt,2
bdf-usecase3-tiny,/home/gerardo/bdf-pz/testdata/bdf-usecase3-tiny,4
sigmod-tiny,/home/gerardo/bdf-pz/testdata/sigmod-tiny,2
sigmod-demo,/home/gerardo/bdf-pz/testdata/sigmod-demo,11
bdf-demo,/home/gerardo/bdf-pz/testdata/bdf-demo,11


Show me the files available in bdf-demo

In [3]:
import os
files = os.listdir("/home/gerardo/bdf-pz/testdata/bdf-usecase3-tiny")

files

['bolderson2010.pdf',
 'bonfiglio2023.pdf',
 'brunyanszki2014.pdf',
 'ceccaldi2015.pdf']

Create a schema to extract authors.

In [1]:
# Define a class name
class_name = "GeneMutationTumorRelation"

gene_mutations = [{'name': 'gene_name',
  'desc': 'The name of the gene mentioned in the text.',
  'type': str},
 {'name': 'disease_name',
  'desc': 'The name of the disease associated with the gene.',
  'type': str},
 {'name': 'context_info',
  'desc': 'Additional context information regarding the gene and disease.',
  'type': str}]

Consider the bdf-demo dataset. This is a dataset of scientific papers. Extract all authors of papers about brain cancer.

In [3]:
import palimpzest as pz
dataset = pz.Dataset("testdata/sigmod-tiny-txt")

# condition = "The paper is about brain cancer."
# dataset = dataset.sem_filter(condition)

convert_schema = gene_mutations
cardinality = pz.Cardinality.ONE_TO_MANY
dataset = dataset.sem_add_columns(convert_schema, cardinality=cardinality)

output = dataset
policy = pz.MinCost()
# policy = pz.MaxQuality()

config = pz.QueryProcessorConfig(
    policy=policy,
    cache=False,
    execution_strategy="parallel",
)

results = output.run(config)
results_df = results.to_df()

Output()

PROMPT:
You are a helpful assistant whose job is to generate a JSON object.
You will be presented with a context and a set of output fields to generate. Your task is to generate a JSON object
which fills in the output fields with the correct values.
You will be provided with a description of each input field and each output field. All of the fields in the output 
JSON object can be derived using information from the context.

Remember, your answer must be a valid JSON list of dictionaries. The list may contain one or more dictionaries, and
each dictionary should only have the specified output fields. Finish your response with a newline character 
followed by ---
---
INPUT FIELDS:
- contents: The contents of the file
- filename: The UNIX-style name of the file

OUTPUT FIELDS:
- context_info: Additional context information regarding the gene and disease.
- disease_name: The name of the disease associated with the gene.
- gene_name: The name of the gene mentioned in the text.

CONTEXT:
{
  "contents": "Resource\nProteogenomic Analysis of Human Colon Cancer\nReveals New Therapeutic Opportunities\nSuhas
Vasaikar,1,2,14 Chen Huang,1,2,14 Xiaojing Wang,1,2,12,14 Vladislav A. Petyuk,3,14Sara R. Savage,4,14Bo 
Wen,1,2\nYongchao Dou,1,2Yun Zhang,1Zhiao Shi,1,2Osama A. Arshad,3Marina A. Gritsenko,3Lisa J. Zimmerman,5\nJason 
E. McDermott,3Therese R. Clauss,3Ronald J. Moore,3Rui Zhao,3Matthew E. Monroe,3Yi-Ting Wang,3\nMatthew C. 
Chambers,5Robbert J.C. Slebos,5Ken S. Lau,6Qianxing Mo,7,13Li Ding,8Matthew Ellis,1,7\nMathangi 
Thiagarajan,9Christopher R. Kinsinger,10Henry Rodriguez,10Richard D. Smith,3Karin D. Rodland,3,11,15, *\nDaniel C. 
Liebler,5,15,*Tao Liu,3,15,*Bing Zhang,1,2,7,15,16, *and Clinical Proteomic Tumor Analysis Consortium\n1Lester and 
Sue Smith Breast Center, Baylor College of Medicine, Houston, TX 77030, USA\n2Department of Molecular and Human 
Genetics, Baylor College of Medicine, Houston, TX 77030, USA\n3Biological Sciences Division, Paci\ufb01c Northwest 
National Laboratory, Richland, WA 99352, USA\n4Department of Biomedical Informatics, Vanderbilt University Medical 
Center, Nashville, TN 37232, USA\n5Department of Biochemistry, Vanderbilt University, Nashville, TN 37232, 
USA\n6Department of Cell and Developmental Biology, Vanderbilt University, Nashville, TN 37232, USA\n7Dan L Duncan 
Comprehensive Cancer Center, Baylor College of Medicine, Houston, TX 77030, USA\n8The McDonnell Genome Institute, 
Washington University in St. Louis, Forest Park Avenue, Campus Box 8501, St. Louis, MO 63108, USA\n9Leidos 
Biomedical Research Inc., Frederick National Laboratory for Cancer Research, Frederick, MD 21702, USA\n10Of\ufb01ce
of Cancer Clinical Proteomics Research, National Cancer Institute, Bethesda, MD 20892, USA\n11Department of Cell, 
Developmental, and Cancer Biology, Oregon Health & Science University, Portland, OR 97221, USA\n12Present address: 
Department of Epidemiology and Biostatistics, Greehey Children\u2019s Cancer Research Institute, The University of 
Texas\nHealth Science Center at San Antonio, San Antonio, TX 78229, USA\n13Present address: Department of 
Biostatistics & Bioinformatics, H. Lee Mof\ufb01tt Cancer Center & Research Institute, Tampa, FL 33612, 
USA\n14These authors contributed equally\n15Senior author\n16Lead Contact\n*Correspondence: karin.rodland@pnnl.gov 
(K.D.R.), daniel.liebler@vanderbilt.edu (D.C.L.), tao.liu@pnnl.gov (T.L.), bing.zhang@bcm.edu 
(B.Z.)\nhttps://doi.org/10.1016/j.cell.2019.03.030\nSUMMARY\nWe performed the \ufb01rst proteogenomic study on\na 
prospectively collected colon cancer cohort.Comparative proteomic and phosphoproteomic\nanalysis of paired tumor 
and normal adjacent tis-\nsues produced a catalog of colon cancer-associ-ated proteins and phosphosites, including 
knownand putative new biomarkers, drug targets, andcancer/testis antigens. Proteogenomic integrationnot only 
prioritized genomically inferred targets,such as copy-number drivers and mutation-derivedneoant

[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m

[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m

[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m
[0m

...

[0m
[0m
[0m

[0m
[0m

[0m

Total time: 8.06s
Total cost: $0.0000


RecursionError: maximum recursion depth exceeded

In [18]:
results_df

,contents,filename,context_info,disease_name,gene_name
0,Resource\nProteogenomic Analysis of Human Colo...,vasaikar2019.txt,None,None,None
1,ARTICLE OPEN\nTowards precision oncology disco...,liu2024.txt,None,None,None


In [9]:
print(stats.total_plan_time)

NameError: name 'stats' is not defined